In [1]:
import numpy as np
from scipy.linalg import norm
from scipy.sparse.linalg import expm_multiply, expm

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp, Statevector

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Estimator 

from qiskit import transpile

from qiskit.providers.fake_provider import FakeManilaV2
from qiskit_aer import AerSimulator

In [2]:
import sys
import os
parentdir = os.path.dirname(os.getcwd())
sys.path.append(parentdir)

In [3]:
try:
    service = QiskitRuntimeService(channel="ibm_quantum")
except:
    print("\n ... Saving ibm_quantum account ... \n")

    token = "f9b97988a51c6d1b71630fc1699f80b6143d73c28ab066c1d6d32ac189848318614ffd2ac8c416599940cfe6307f5eea61d38be05a55942056c8a235f4275d73"

    QiskitRuntimeService.save_account(
        channel="ibm_quantum", instance="ibm-q/open/main", token=(token), overwrite=True
    )
    service = QiskitRuntimeService(channel="ibm_quantum")


In [4]:
#backend = service.backend("ibm_brisbane")
backend = "ibmq_qasm_simulator"

session = Session(service=service, backend=backend)

estimator = Estimator(
    session=session,
    options={
        "resilience_level": 1, # readout error mitigation
        "transpilation": {
            "skip_transpilation": True, # we have already transpiled our circuits
        },
    }
)

In [5]:
# definitions for circuit
def get_H_op(N, J, h, pbc):
    """Define the two non-commuting parts of the Ising Hamiltonian separately."""

    z_op_strings = []
    for j in range(0, N - 1, 2):
        z_op_strings.append((N - 2 - j) * "I" + "Z" + "Z" + j * "I")
    for j in range(1, N - 1, 2):
        z_op_strings.append((N - 2 - j) * "I" + "Z" + "Z" + j * "I")
    if pbc and N > 2:
        z_op_strings.append("Z" + (N - 2) * "I" + "Z")

    x_op_strings = []
    for i in range(N):
        x_op_strings.append(((N - 1 - i) * "I" + "X" + i * "I"))

    z_ops = SparsePauliOp(
        data=z_op_strings, coeffs=[J] * (N if (pbc and N > 2) else N - 1)
    )
    x_ops = SparsePauliOp(data=x_op_strings, coeffs=[h] * N)

    return z_ops, x_ops

def trotter_circ(N, H_list, t_final, dt):
    Nt = int(t_final / dt)
    init_circ = QuantumCircuit(N)

    # ZZ-gate
    z_circ = PauliEvolutionGate(H_list[0], dt)
    x_circ = PauliEvolutionGate(H_list[1], dt)

    qreg = QuantumRegister(N)
    evo_circ = QuantumCircuit(qreg)
    evo_circ.append(init_circ, qreg)
    for _ in range(Nt):
        evo_circ.append(x_circ, qreg)
        evo_circ.append(z_circ, qreg)

    return evo_circ


In [6]:
# Creating the actual circuit
N = 4
J = 1
h = 0.7
pbc = False

t_final = 4
Nt = 10
dt = t_final / Nt
t_eval = np.linspace(0, t_final, Nt + 1)

# Our observable
mag_ave = SparsePauliOp(
    data=[((N - 1 - i) * "I" + "Z" + i * "I") for i in range(N)], coeffs=[1 / N] * N
)

initial_layout = [112, 126, 125, 124]  # physical qubits on the device
assert(len(initial_layout) == N)
shots = 10000

z_ops, x_ops = get_H_op(N, J, h, pbc)
H_op = z_ops + x_ops
print(f"H_Z =\n{z_ops}")
print(f"H_X =\n{x_ops}")


# construct and transpile all Trotter circuits (for each time step)
t_circs = []
for t in t_eval:
    evo_circ = trotter_circ(N, [z_ops, x_ops], t_final=t, dt=dt)
    tc = transpile(
        evo_circ,
        backend = service.backend("ibm_brisbane"),
        initial_layout=initial_layout,
        optimization_level=3,
    )
    if not t == 0:
        print(
            f"t = {t}, "
            f"#2q-gates={tc.count_ops()['ecr']}, "
            f"depth={tc.depth()}, "
            f"2q-depth={tc.depth(lambda circ_instruct: True if circ_instruct[0].name == 'ecr' else False)}"
        )

    t_circs.append(tc)

# transpile the observable to match the physical qubits
t_obs = mag_ave.apply_layout(t_circs[0].layout)


H_Z =
SparsePauliOp(['IIZZ', 'ZZII', 'IZZI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j])
H_X =
SparsePauliOp(['IIIX', 'IIXI', 'IXII', 'XIII'],
              coeffs=[0.7+0.j, 0.7+0.j, 0.7+0.j, 0.7+0.j])
t = 0.4, #2q-gates=6, depth=27, 2q-depth=4
t = 0.8, #2q-gates=12, depth=51, 2q-depth=8
t = 1.2000000000000002, #2q-gates=18, depth=74, 2q-depth=12
t = 1.6, #2q-gates=24, depth=97, 2q-depth=16
t = 2.0, #2q-gates=30, depth=120, 2q-depth=20
t = 2.4000000000000004, #2q-gates=36, depth=143, 2q-depth=24
t = 2.8000000000000003, #2q-gates=42, depth=166, 2q-depth=28
t = 3.2, #2q-gates=48, depth=189, 2q-depth=32
t = 3.6, #2q-gates=54, depth=212, 2q-depth=36
t = 4.0, #2q-gates=60, depth=235, 2q-depth=40


In [7]:
from qhub_client import QhubClient
client = QhubClient()


In [8]:

job = estimator.run(
    circuits=t_circs,
    observables=[t_obs] * len(t_circs),
    shots=shots
)

client.benchmarq(job)

print("")
print("Job ID: ", job.job_id())

print("Inputs: ", job.inputs)

job.wait_for_final_state()
result = job.result()
print("Result: ", result)

print("Metrics: ", job.metrics())


result = job.result()
mag_device = result.values
print(f"Result: {mag_device}")





{'job_id': 'cm5ldjoiidfp3m8mjh00', 'result': {'results': [1.0, 0.84315, 0.59855, 0.4479, 0.37005, 0.34735, 0.388, 0.4268, 0.42685000000000006, 0.405, 0.40159999999999996]}, 'metrics': {'timestamps': {'created': '2023-12-26T22:42:23.355874Z', 'finished': '2023-12-26T22:42:26.943824Z', 'running': '2023-12-26T22:42:23.432403Z'}, 'bss': {'seconds': 0}, 'usage': {'quantum_seconds': 0, 'seconds': 0}, 'executions': 0, 'num_circuits': 0, 'qiskit_version': 'qiskit_ibm_runtime-0.17.0,qiskit_terra-0.45.1,qiskit_aer-0.13.1*', 'caller': 'qiskit_ibm_runtime~estimator.py'}}
{"message":"Algorithm added to Qhub. \n\n    Check out your coontribution at: http://localhost:3000/api/algs"}

Job ID:  cm5ldjoiidfp3m8mjh00
Inputs:  {'circuits': [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14589b610>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x145577b50>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14c6708e0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1

In [9]:
print(result.values)
print(type(result.values))

res = {"results": result.values.tolist()}
print(res)
print(type(res))
print(type(job.metrics()))

[1.      0.84315 0.59855 0.4479  0.37005 0.34735 0.388   0.4268  0.42685
 0.405   0.4016 ]
<class 'numpy.ndarray'>
{'results': [1.0, 0.84315, 0.59855, 0.4479, 0.37005, 0.34735, 0.388, 0.4268, 0.42685000000000006, 0.405, 0.40159999999999996]}
<class 'dict'>
<class 'dict'>
